# Project 03: Decision Tree


## Import Libraries


In [ ]:
import graphviz
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz


In [ ]:
from constants.path import DATA_CARS, DATA_WDBC, DATA_WINE, FIG_DIR
from constants.base import RATIOS
from utils import display_ratio, normalize_dataset_name, normalize_dataset_ratio


## Dataset Loading and Preprocessing


In [ ]:
breast_df = pd.read_csv(DATA_WDBC, sep=",")
# Map labels (M=0, B=1) to align with sklearn's encoding
breast_df = breast_df[breast_df["Diagnosis"].isin(["M", "B"])]

# Prepare features and labels
y_breast = breast_df["Diagnosis"].map({"M": 0, "B": 1})
X_breast = breast_df.drop("Diagnosis", axis=1)

# Verify dataset integrity
# print("Unique Diagnosis values:", breast_df["Diagnosis"].unique())
# print("Unique y_breast:", y_breast.unique())
# print("Number of features:", X_breast.shape[1])
# print("Feature Names:\n", len(X_breast.columns.tolist()), (X_breast.columns.tolist()))
# print("\nClass Distribution:\n", y_breast.value_counts())


In [ ]:
wine_df = pd.read_csv(DATA_WINE, sep=",")
# Group into 3 quality categories
wine_df["quality_group"] = pd.cut(
    wine_df["quality"], bins=[-1, 4, 6, 10], labels=["Low", "Standard", "High"]
)

# Prepare features and labels
y_wine = wine_df["quality_group"]
X_wine = wine_df.drop(["quality", "quality_group"], axis=1)

# Verify class distribution
# print("Quality Group Distribution:")
# print(y_wine.value_counts())
# print("Unique y_wine:", y_wine.unique())


In [ ]:
"""
Additional Dataset: Car Evaluation Dataset

This dataset meets all project requirements:
	+ Source: UCI Machine Learning Repository
	+ Classes: 4 (unacc, acc, good, vgood)
	+ Samples: 1728

Features: 6 categorical attributes (buying price, maintenance cost, doors, etc.).
"""


car_df = pd.read_csv(DATA_CARS, sep=",")

# One-Hot Encode Categorical Features
y_car = car_df["class"]
X_car = pd.get_dummies(car_df.drop("class", axis=1))

# Verify Dataset Compliance
# print(f"Total Samples: {len(car_df)}")  # Output: 1728
# print("Class Distribution:\n", y_car.value_counts())
# print("class names:\n", y_car.unique())


## 2.1 - Data Splitting and Visualization


In [ ]:
def split_and_visualize(X, y, dataset_name: str):
    splits = {}

    for ratio in RATIOS:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, train_size=ratio, stratify=y, shuffle=True, random_state=42
        )

        splits[ratio] = (X_train, X_test, y_train, y_test)

        # Class distribution visualization
        _, axes = plt.subplots(1, 3, figsize=(18, 4))
        (
            pd.Series(y)
            .value_counts(normalize=True)
            .plot(kind="bar", ax=axes[0], title=f"Original ({len(y)} samples)")
        )
        (
            pd.Series(y_train)
            .value_counts(normalize=True)
            .plot(kind="bar", ax=axes[1], title=f"Train ({len(y_train)} samples)")
        )
        (
            pd.Series(y_test)
            .value_counts(normalize=True)
            .plot(kind="bar", ax=axes[2], title=f"Test ({len(y_test)} samples)")
        )

        plt.suptitle(f"{dataset_name} - {display_ratio(ratio)} Split")
        plt.tight_layout()
        plt.savefig(
            f"{FIG_DIR}/class_dist/class_dist__{normalize_dataset_name(dataset_name)}__{normalize_dataset_ratio(ratio)}.png"
        )
        plt.show()
        plt.close()

    return splits


## 2.2, 2.3 - Decision Tree Training and Evaluation


In [ ]:
def train_evaluate_decision_tree(
    X_train,
    y_train,
    X_test,
    y_test,
    dataset_name: str,
    split_ratio,
):
    # Dynamically set feature and class names
    feature_names = X_train.columns.tolist()
    class_names = [str(cls) for cls in np.unique(y_train)]

    # Train model
    clf = DecisionTreeClassifier(criterion="entropy", random_state=42)
    clf.fit(X_train, y_train)

    # Predictions
    y_pred = clf.predict(X_test)

    # Classification Report (with validation)
    print(f"\nClassification Report ({dataset_name}, {display_ratio(split_ratio)}):")
    print(
        classification_report(
            y_test,
            y_pred,
            target_names=class_names,
            labels=np.unique(y_test),  # Ensure alignment with actual classes
        )
    )

    sns.heatmap(
        confusion_matrix(y_test, y_pred),
        annot=True,
        fmt="d",
        xticklabels=class_names,
        yticklabels=class_names,
    )
    plt.title(f"Confusion Matrix - {dataset_name} ({display_ratio(split_ratio)})")
    plt.savefig(
        f"{FIG_DIR}/confusion_mat/confusion_mat__{normalize_dataset_name(dataset_name)}__{normalize_dataset_ratio(split_ratio)}.png"
    )
    plt.show()
    plt.close()

    # Export Decision Tree Visualization
    dot_data = export_graphviz(
        clf,
        out_file=None,
        feature_names=feature_names,
        class_names=class_names,
        filled=True,
        rounded=True,
    )
    img = mpimg.imread(
        graphviz.Source(dot_data).render(
            f"{FIG_DIR}/dt/dt__{normalize_dataset_name(dataset_name)}__{normalize_dataset_ratio(split_ratio)}",
            format="png",
            cleanup=True,
        )
    )
    height, width = img.shape[:2]
    dpi = plt.rcParams["figure.dpi"]
    _, ax = plt.subplots(figsize=(width / dpi, height / dpi))
    ax.imshow(img)
    ax.axis("off")

    plt.title(f"Decision Tree - {dataset_name} ({display_ratio(split_ratio)})")
    plt.show()
    plt.close()


## 2.4 - Depth vs. Accuracy Analysis (80/20 Split)


In [ ]:
def depth_accuracy_analysis(X, y, feature_names, class_names, dataset_name: str):
    # 80/20 split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, train_size=0.8, stratify=y, shuffle=True, random_state=42
    )

    # Test depths
    max_depths = [None, 2, 3, 4, 5, 6, 7]
    accuracies = []

    for depth in max_depths:
        # Train with depth constraint
        clf = DecisionTreeClassifier(
            criterion="entropy", max_depth=depth, random_state=42
        )
        clf.fit(X_train, y_train)
        acc = accuracy_score(y_test, clf.predict(X_test))
        accuracies.append(acc)

        dot_data = export_graphviz(
            clf,
            out_file=None,
            feature_names=feature_names,
            class_names=class_names,
            filled=True,
            rounded=True,
        )
        img = mpimg.imread(
            graphviz.Source(dot_data).render(
                f"{FIG_DIR}/dt/dt__{normalize_dataset_name(dataset_name)}__{normalize_dataset_ratio(0.8)}__{depth}",
                format="png",
                cleanup=True,
            )
        )
        height, width = img.shape[:2]
        dpi = plt.rcParams["figure.dpi"]
        _, ax = plt.subplots(figsize=(width / dpi, height / dpi))
        ax.imshow(img)
        ax.axis("off")

        plt.title(f"Decision Tree - {dataset_name} (80/20) (depth={depth})")
        plt.show()
        plt.close()

    # Accuracy vs Depth Plot
    plt.title(f"{dataset_name} - Accuracy vs. Tree Depth (80/20 Split)")
    plt.plot([str(d) if d else "Unlimited" for d in max_depths], accuracies, marker="o")
    plt.xlabel("Max Tree Depth")
    plt.ylabel("Test Accuracy")
    plt.grid(linestyle="--")
    plt.savefig(
        f"{FIG_DIR}/accuracy_vs_depth_{normalize_dataset_name(dataset_name)}.png"
    )
    plt.show()
    plt.close()

    # Return results as DataFrame
    return pd.DataFrame(
        {
            "Max Depth": [x if x is not None else "None" for x in max_depths],
            "Accuracy": accuracies,
        }
    )


## Training and Visualization


In [ ]:
import os

os.makedirs(FIG_DIR, exist_ok=True)
os.makedirs(f"{FIG_DIR}/class_dist", exist_ok=True)
os.makedirs(f"{FIG_DIR}/confusion_mat", exist_ok=True)
os.makedirs(f"{FIG_DIR}/dt", exist_ok=True)


### Breast Cancer


In [ ]:
dataset_name = "Breast Cancer"

splitted_tests = split_and_visualize(X_breast, y_breast, dataset_name)
for ratio in RATIOS:
    X_train, X_test, y_train, y_test = splitted_tests[ratio]
    train_evaluate_decision_tree(
        X_train,
        y_train,
        X_test,
        y_test,
        dataset_name,
        ratio,
    )

feature_names = X_breast.columns.tolist()
class_names = ["Malignant", "Benign"]  # Must match y_breast.unique()
depth_results = depth_accuracy_analysis(
    X_breast, y_breast, feature_names, class_names, dataset_name
)
print(f"\nDepth Analysis Results for {dataset_name}:\n", depth_results)


### Wine Quality


In [ ]:
dataset_name = "Wine Quality"

splitted_tests = split_and_visualize(X_wine, y_wine, dataset_name)
for ratio in RATIOS:
    X_train, X_test, y_train, y_test = splitted_tests[ratio]
    train_evaluate_decision_tree(
        X_train,
        y_train,
        X_test,
        y_test,
        dataset_name,
        ratio,
    )

feature_names = X_wine.columns.tolist()
class_names = ["Standard", "High", "Low"]  # Must match y_wine.unique()
depth_results = depth_accuracy_analysis(
    X_wine, y_wine, feature_names, class_names, dataset_name
)
print(f"\nDepth Analysis Results for {dataset_name}:\n", depth_results)


### Car Evaluation


In [ ]:
dataset_name = "Car Evaluation"

splitted_tests = split_and_visualize(X_car, y_car, dataset_name)
for ratio in RATIOS:
    X_train, X_test, y_train, y_test = splitted_tests[ratio]
    train_evaluate_decision_tree(
        X_train,
        y_train,
        X_test,
        y_test,
        dataset_name,
        ratio,
    )

feature_names = X_car.columns.tolist()
class_names = ["unacc", "acc", "vgood", "good"]  # Must match y_car.unique()
depth_results = depth_accuracy_analysis(
    X_car, y_car, feature_names, class_names, dataset_name
)
print(f"\nDepth Analysis Results for {dataset_name}:\n", depth_results)
